In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt

In [2]:
train_df = pd.read_csv(r"C:\Users\kammari.vara.prasad\Documents\Test\train_data\train.csv")

In [3]:
train_df.shape

(404290, 6)

In [4]:
train_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [5]:
train_df.loc[0].question1

'What is the step by step guide to invest in share market in india?'

In [6]:
train_df.loc[0].question2

'What is the step by step guide to invest in share market?'

In [7]:
train_df['is_duplicate'].value_counts()

0    255027
1    149263
Name: is_duplicate, dtype: int64

In [8]:
dup_df= train_df[train_df['is_duplicate']==1]
non_dup_df = train_df[train_df['is_duplicate']==0]

In [9]:
dup_df.reset_index()
non_dup_df.reset_index()

,index,id,qid1,qid2,question1,question2,is_duplicate
0,0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...,...,...,...
255022,404283,404283,537924,537925,What do you think of the removal of the MagSaf...,What will the CPU upgrade to the 2016 Apple Ma...,0
255023,404285,404285,433578,379845,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
255024,404287,404287,537928,537929,What is one coin?,What's this coin?,0
255025,404288,404288,537930,537931,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0


In [26]:
#!pip install -U sentence-transformers

In [10]:
import os
os.environ['CURL_CA_BUNDLE'] = ''

In [11]:
from sentence_transformers import CrossEncoder
model = CrossEncoder('cross-encoder/quora-roberta-large')


C:\Users\kammari.vara.prasad\AppData\Roaming\Python\Python39\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\kammari.vara.prasad\AppData\Roaming\Python\Python39\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [ ]:

#Cross Encoder
#Cross-Encoders can be used whenever you have a pre-defined set of sentence pairs you want to score.
#For example, you have 100 sentence pairs and you want to get similarity scores for these 100 pairs.
#Cross Encoder is not suitable for our duplicate detection task

In [17]:
q2='RRA and RDB AI Validation for Story entity'
q3='RRA and RSP AI Validation for Story entity'

In [18]:
scores = model.predict([(q3,q2)])
scores

array([0.01103877], dtype=float32)

In [19]:
# From the above pairs , we can see that model is performing poor on unseen data 

In [20]:
import os
os.environ['CURL_CA_BUNDLE'] = ''
import warnings
warnings.filterwarnings('ignore') # setting ignore as a parameter

In [21]:
from sentence_transformers import SentenceTransformer, models

## Step 1: use an existing language model
word_embedding_model = models.Transformer('distilroberta-base')

## Step 2: use a pool function over the token embeddings
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

## Join steps 1 and 2 using the modules argument
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

C:\Users\kammari.vara.prasad\AppData\Roaming\Python\Python39\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassificati

KeyboardInterrupt: 